In [75]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [76]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate, \
    HumanMessagePromptTemplate, SystemMessagePromptTemplate
from langchain.schema import SystemMessage
from langchain.chat_models.openai import ChatOpenAI
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from pathlib import Path

from htools import *
from jabberwocky.openai_utils import *

In [77]:
cd_root()

Current directory: /Users/hmamin/roboduck


In [78]:
os.environ['OPENAI_API_KEY'] = api_key = load_openai_api_key()

In [79]:
print(load_prompt('debug')['prompt'])

debug: Could try davinci text as well but codex is free for now. You may want to strip triple double-quotes from the end in case codex generates them (we don't include that as a stop phrase because codex might generate a docstring as part of a correct code snippet).
-------------------------------------------------------------------------------

"""ANSWER KEY

This code snippet is not working as expected. Help me debug it. First read my question, then examine the snippet of code that is causing the issue and look at the values of the local and global variables. In the section titled SOLUTION PART 1, use plain English to explain what the problem is and how to fix it. In the section titled SOLUTION PART 2, write a corrected version of the input code snippet. If you don't know what the problem is, SOLUTION PART 1 should list a few possible causes or things I could try in order to identify the issue and SOLUTION PART 2 should say N/A. Be concise and use simple language because I am a begin

In [80]:
system_prompt_text = """You are an incredibly effective AI programming assistant. You have in-depth knowledge across a broad range of sub-fields within computer science, software development, and data science, and your goal is to help Python programmers resolve their most challenging bugs.
"""
system_prompt = SystemMessage(content=system_prompt_text)

In [81]:
user_prompt_text = """This code snippet is not working as expected. Help me debug it. First read my question, then examine the snippet of code that is causing the issue and look at the values of the local and global variables. Your response must have exactly two parts. In the section titled SOLUTION PART 1, use plain English to explain what the problem is and how to fix it (if you don't know what the problem is, SOLUTION PART 1 should instead list a few possible causes or things I could try in order to identify the issue). In the section titled SOLUTION PART 2, write a corrected version of the input code snippet (if you don't know, SOLUTION PART 2 should say None). SOLUTION PART 2 must contain only python code - there must not be any English explanation outside of code comments or docstrings. Be concise and use simple language because I am a beginning programmer.

QUESTION:
{question}

CURRENT CODE SNIPPET:
{code}

LOCAL VARIABLES:
{local_vars}

GLOBAL VARIABLES:
{global_vars}"""
user_prompt_template = HumanMessagePromptTemplate.from_template(
    user_prompt_text
)

In [82]:
kwargs = {
    'question': 'Why will this throw an index error soon?',
    'code': """def bubble_sort(nums):
    for i in range(len(nums)):
        for j in range(len(nums)):
            if nums[j] > nums[j + 1]:
                nums[j + 1], nums[j] = nums[j], nums[j + 1]
    return nums""",
    'local_vars': """{
    'nums': [3, 4, 2, 1, 5, 9],   # type: list
    'i': 0,   # type: int
    'j': 4,   # type: int
}""",
    'global_vars': """{
}"""
}

In [83]:
messages = [
    system_prompt,
    user_prompt_template.format(**kwargs)
]

In [84]:
print('\n'.join(m.content for m in messages))

You are an incredibly effective AI programming assistant. You have in-depth knowledge across a broad range of sub-fields within computer science, software development, and data science, and your goal is to help Python programmers resolve their most challenging bugs.

This code snippet is not working as expected. Help me debug it. First read my question, then examine the snippet of code that is causing the issue and look at the values of the local and global variables. Your response must have exactly two parts. In the section titled SOLUTION PART 1, use plain English to explain what the problem is and how to fix it (if you don't know what the problem is, SOLUTION PART 1 should instead list a few possible causes or things I could try in order to identify the issue). In the section titled SOLUTION PART 2, write a corrected version of the input code snippet (if you don't know, SOLUTION PART 2 should say None). SOLUTION PART 2 must contain only python code - there must not be any English ex

In [88]:
chat = ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo')

In [58]:
res = chat(messages)
print(res.content)

SOLUTION PART 1:
The code will throw an index error soon because the inner loop is iterating up to the length of the list, which means that on the last iteration, `nums[j + 1]` will be out of range. To fix this, we need to change the range of the inner loop to `range(len(nums) - i - 1)`.

SOLUTION PART 2:

```
def bubble_sort(nums):
    for i in range(len(nums)):
        for j in range(len(nums) - i - 1):
            if nums[j] > nums[j + 1]:
                nums[j + 1], nums[j] = nums[j], nums[j + 1]
    return nums
```


In [60]:
messages.append(res)
messages.append(
    user_prompt_template.format(
        **{**kwargs, 
           'question': 'Can you revise your solution so you only find the length of nums once?'}
    )
)

In [62]:
res = chat(messages)
print(res.content)

SOLUTION PART 1:
The problem with the current code is that it is calling `len(nums)` twice in the inner loop, which is inefficient. To fix this, we can store the length of `nums` in a variable before the loop and use that variable instead.

SOLUTION PART 2:

```
def bubble_sort(nums):
    n = len(nums)
    for i in range(n):
        for j in range(n - i - 1):
            if nums[j] > nums[j + 1]:
                nums[j + 1], nums[j] = nums[j], nums[j + 1]
    return nums
```


Took first stab at storing this info in a file (py for now). Try loading.

In [165]:
import importlib
from langchain.schema import AIMessage
from roboduck.prompts.chat import debug

In [194]:
class Chat:
    
    def __init__(self, system_text, user_text, model_kwargs=None, history=()):
        # TODO: create new Chat obj or let user pass in or configure based on
        # user kwargs? For now just return dummy message.
        self.chat = lambda x: AIMessage(content=x[-1].content.upper())
        # TODO: may need to adjust some kwarg names to match langchain UX.
        # Need to look into when/how I'm supposed to provide these - currently
        # doing nothing with them. Look at ChatOpenai constructor.
        self.kwargs = dict(model_kwargs or {})
        self.system_message = SystemMessage(content=system_text)
        if isinstance(user_text, str):
            user_text = {'reply': user_text}
        self.user_templates = {
            k: HumanMessagePromptTemplate.from_template(v)
            for k, v in user_text.items()
        }
        self.default_user_key = next(iter(self.user_templates))
        self._history = list(history) or [self.system_message]
        
    @classmethod
    def load_template(cls, prompt_name):
        module = importlib.import_module(
            f'roboduck.prompts.chat.{prompt_name}'
        )
        return cls(module.system, module.user, module.kwargs)
        
    def _user_message(self, *, key='', **kwargs):
        key = key or self.default_user_key
        template = self.user_templates[key]
        return template.format(**kwargs)
    
    def reply(self, *, key='', **kwargs):
        user_message = self._user_message(key=key, **kwargs)
        self._history.append(user_message)
        try:
            response = self.chat(self._history)
        except Exception as e:
            self._history.pop(-1)
            raise e
        self._history.append(response)
        return response
    
    def history(self, sep='\n\n'):
        """Return chat history as a single string."""
        res = []
        for row in self._history:
            name = type(row).__name__.split('Message')[0]
            res.append(f'{name}: {row.content}')
        return sep.join(res)

In [184]:
chat = Chat.load_template('debug')

In [185]:
tmp = chat._user_message(
    code='a = 3\nb = 4', question='Why?', local_vars='{3: 4}',
    global_vars='{True: False}', next_line='b = 4'
)

In [186]:
print(tmp.content)

This code snippet is not working as expected. Help me debug it. First read my question, then examine the snippet of code that is causing the issue and look at the values of the local and global variables. Your response must have exactly two parts. In the section titled SOLUTION PART 1, use plain English to explain what the problem is and how to fix it (if you don't know what the problem is, SOLUTION PART 1 should instead list a few possible causes or things I could try in order to identify the issue). In the section titled SOLUTION PART 2, write a corrected version of the input code snippet (if you don't know, SOLUTION PART 2 should say None). SOLUTION PART 2 must contain only python code - there must not be any English explanation outside of code comments or docstrings. Be concise and use simple language because I am a beginning programmer.

QUESTION:
Why?

CURRENT CODE SNIPPET:
a = 3
b = 4

NEXT LINE:
b = 4

LOCAL VARIABLES:
{3: 4}

GLOBAL VARIABLES:
{True: False}


In [187]:
tmp = chat._user_message(
    key='contextless',
    question='Why?'
)

In [188]:
print(tmp.content)

QUESTION:
Why?


In [189]:
chat.reply(key='contextless', question='How are you?')

AIMessage(content='QUESTION:\nHOW ARE YOU?', additional_kwargs={})

In [190]:
tmp = chat.history()
print(tmp)

System: You are an incredibly effective AI programming assistant. You have in-depth knowledge across a broad range of sub-fields within computer science, software development, and data science, and your goal is to help Python programmers resolve their most challenging bugs.

Human: QUESTION:
How are you?

AI: QUESTION:
HOW ARE YOU?


## Debug scratch

See if we can use frames to identify whether we need to provide context for a user message (i.e. if frame has changed since we last did).

In [4]:
from roboduck.debugger import duck

In [70]:
def binary_search(x, nums):
    if not nums:
        return -1
    duck(backend='repeat')
    mid = len(nums) // 2
    if x == nums[mid]:
        return x
    if x > nums[mid]:
        return binary_search(x, nums[mid + 1:])
    if x < nums[mid]:
        return binary_search(x, nums[:mid])

In [71]:
nums = [33, 44, 55, 66, 77, 88, 99, 111]

In [72]:
binary_search(3, nums)

> <ipython-input-70-8a37149461d4>(5)binary_search()
-> mid = len(nums) // 2
>>> l .
  1  	def binary_search(x, nums):
  2  	    if not nums:
  3  	        return -1
  4  	    duck(backend='repeat')
  5  ->	    mid = len(nums) // 2
  6  	    if x == nums[mid]:
  7  	        return x
  8  	    if x > nums[mid]:
  9  	        return binary_search(x, nums[mid + 1:])
 10  	    if x < nums[mid]:
 11  	        return binary_search(x, nums[:mid])
>>> y?
next line:     mid = len(nums) // 2
[Duck] """>>> n
> <ipython-input-70-8a37149461d4>(6)binary_search()
-> if x == nums[mid]:
>>> n
> <ipython-input-70-8a37149461d4>(8)binary_search()
-> if x > nums[mid]:
>>> n
> <ipython-input-70-8a37149461d4>(10)binary_search()
-> if x < nums[mid]:
>>> n
> <ipython-input-70-8a37149461d4>(11)binary_search()
-> return binary_search(x, nums[:mid])
>>> l .
  6  	    if x == nums[mid]:
  7  	        return x
  8  	    if x > nums[mid]:
  9  	        return binary_search(x, nums[mid + 1:])
 10  	    if x < nums[mid

BdbQuit: 

In [13]:
binary_search(33, nums)

33

In [14]:
binary_search(39, nums)

-1

In [15]:
binary_search(111, nums)

111

In [16]:
binary_search(112, nums)

-1

In [73]:
def test():
    for i in range(5):
        print(i)
        duck(backend='repeat')

In [74]:
test()

0
> <ipython-input-73-943befe6b744>(2)test()
-> for i in range(5):
>>> i
0
>>> l .
  1  	def test():
  2  ->	    for i in range(5):
  3  	        print(i)
  4  	        duck(backend='repeat')
[EOF]
>>> y?
frmae_id 140192236901280
next line:     for i in range(5):
[Duck] """>>> n
> <ipython-input-73-943befe6b744>(3)test()
-> print(i)
>>> i
1
>>> y?
frmae_id 140192236901280
next line:         print(i)
[Duck] """>>> n
1
> <ipython-input-73-943befe6b744>(4)test()
-> duck(backend='repeat')
>>> i
1
>>> n
> <ipython-input-73-943befe6b744>(2)test()
-> for i in range(5):
>>> i
1
>>> l .
  1  	def test():
  2  ->	    for i in range(5):
  3  	        print(i)
  4  	        duck(backend='repeat')
[EOF]
>>> i
1
>>> y?
frmae_id 140192236901280
next line:     for i in range(5):
[Duck] """>>> n
> <ipython-input-73-943befe6b744>(3)test()
-> print(i)
>>> l .
  1  	def test():
  2  	    for i in range(5):
  3  ->	        print(i)
  4  	        duck(backend='repeat')
[EOF]
>>> y?
frmae_id 140192236901280


BdbQuit: 